# Project: Book-Inventory-Web-Scraper

## Project Objective
This notebook processes raw data scraped from *Books to Scrape* to build a dataset suitable for a Power BI dashboard.

**Key tasks performed:**
1.  **Data Cleaning:** Handling currency symbols and renaming technical columns.
2.  **Simulation:** Generating synthetic data for *Stock Levels* and *Ratings* (missing from the raw scrape).
3.  **Feature Engineering:** Creating new business KPIs like *Potential Revenue* and *Inventory Status*.

##Step 1: Setup & Import Libraries
We import `pandas` for data manipulation and `numpy` for mathematical operations. We also import `files` from `google.colab` to export our final dataset.

In [2]:
import pandas as pd
import numpy as np
from google.colab import files

print("Libraries imported successfully!")

Libraries imported successfully!


##Step 2: Load the Scraped Data
We load the raw CSV file named `raw_books_data.csv`.
> **Note:** Ensure you have uploaded the file to the Colab "Files" sidebar before running this cell.

In [5]:
# Load the uploaded CSV file
df = pd.read_csv('raw_books_data.csv')

# Show the first 5 rows to understand the structure
print("Raw Data Preview:")
display(df.head())

Raw Data Preview:


,image_container href,thumbnail src,product_pod,price_color
0,https://books.toscrape.com/catalogue/a-light-i...,https://books.toscrape.com/media/cache/2c/da/2...,A Light in the ...,£51.77
1,https://books.toscrape.com/catalogue/tipping-t...,https://books.toscrape.com/media/cache/26/0c/2...,Tipping the Velvet,£53.74
2,https://books.toscrape.com/catalogue/soumissio...,https://books.toscrape.com/media/cache/3e/ef/3...,Soumission,£50.10
3,https://books.toscrape.com/catalogue/sharp-obj...,https://books.toscrape.com/media/cache/32/51/3...,Sharp Objects,£47.82
4,https://books.toscrape.com/catalogue/sapiens-a...,https://books.toscrape.com/media/cache/be/a5/b...,Sapiens: A Brief History ...,£54.23


##Step 3: Rename Columns
The scraping tool generated technical column names (e.g., `product_pod`). We rename these to business-friendly terms like `Title` and `Price` for clarity.

In [6]:
# Rename columns using a dictionary
df = df.rename(columns={
    'product_pod': 'Title',
    'price_color': 'Price',
    'image_container href': 'Product_URL',
    'thumbnail src': 'Image_URL'
})

# Verify the change
print("Columns renamed:")
print(df.columns.tolist())

Columns renamed:
['Product_URL', 'Image_URL', 'Title', 'Price']


##Step 4: Clean 'Price' Column
The `Price` column currently contains the `£` symbol and is stored as text. We remove the symbol and convert the column to a numeric format (float) to enable calculations.

In [7]:
# Remove '£' symbol
df['Price'] = df['Price'].astype(str).str.replace('£', '', regex=False)

# Convert to numeric (float)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

print("Price column cleaned. Example values:")
print(df['Price'].head())

Price column cleaned. Example values:
0    51.77
1    53.74
2    50.10
3    47.82
4    54.23
Name: Price, dtype: float64


##Step 5: Generate Synthetic Data
**Why this step?** The raw web scrape did not capture *Star Ratings* or *Stock Levels*.
To demonstrate dashboard capabilities, we will simulate this data:
* **Category:** Assigned randomly from a list of book genres.
* **Star Rating:** Random integer between 1 and 5.
* **Stock Count:** Weighted random numbers (80% In Stock, 20% Low Stock).

In [8]:
# 1. Create Random Categories
categories = ['Fiction', 'Non-Fiction', 'Science', 'History', 'Fantasy', 'Romance', 'Mystery', 'Business']
df['Category'] = np.random.choice(categories, size=len(df))

# 2. Create Random Star Ratings (1 to 5)
df['Star_Rating'] = np.random.randint(1, 6, size=len(df))

# 3. Create Random Stock Counts
# We want most items to be in stock, but some (20%) to be low on stock to analyze risk.
stock_good = np.random.randint(5, 51, size=int(len(df)*0.8)) # 80% healthy stock
stock_low = np.random.randint(0, 5, size=int(len(df)*0.2))   # 20% critical stock

# Combine and shuffle them so low stock items are scattered randomly
stock_combined = np.concatenate([stock_good, stock_low])
np.random.shuffle(stock_combined)

df['Stock_Count'] = stock_combined

print("Synthetic data generated for: Category, Star_Rating, Stock_Count")

Synthetic data generated for: Category, Star_Rating, Stock_Count


##Step 6: Feature Engineering (KPIs)
We derive new metrics to provide deeper business insights:
1.  **Inventory_Status:** Flags items as *"In Stock"* or *"Low Stock"* (< 5 units).
2.  **Potential_Revenue:** Calculates the total value of stock on hand (`Price` × `Stock_Count`).

In [9]:
# Create a text status for the dashboard
df['Inventory_Status'] = np.where(df['Stock_Count'] > 5, 'In Stock', 'Low Stock')

# Calculate the value of inventory held
df['Potential_Revenue'] = df['Price'] * df['Stock_Count']

# Check the new columns
print(df[['Title', 'Inventory_Status', 'Potential_Revenue']].head())

                          Title Inventory_Status  Potential_Revenue
0            A Light in the ...         In Stock             621.24
1            Tipping the Velvet         In Stock            2095.86
2                    Soumission         In Stock            2254.50
3                 Sharp Objects         In Stock            1291.14
4  Sapiens: A Brief History ...        Low Stock             216.92


##Step 7: Final Check & Export
We review the dataset structure using `.info()` and download the cleaned `cleaned_books_data.csv` file to the local machine for Power BI import.

In [10]:
# 1. Final Check
print("Final Dataset Info:")
print(df.info())

# 2. Save to CSV
output_filename = 'cleaned_books_data.csv'
df.to_csv(output_filename, index=False)
print(f"\nSaved {output_filename} successfully.")

# 3. Trigger Download
files.download(output_filename)

Final Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Product_URL        1000 non-null   object 
 1   Image_URL          1000 non-null   object 
 2   Title              1000 non-null   object 
 3   Price              1000 non-null   float64
 4   Category           1000 non-null   object 
 5   Star_Rating        1000 non-null   int64  
 6   Stock_Count        1000 non-null   int64  
 7   Inventory_Status   1000 non-null   object 
 8   Potential_Revenue  1000 non-null   float64
dtypes: float64(2), int64(2), object(5)
memory usage: 70.4+ KB
None

Saved cleaned_books_data.csv successfully.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>